In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('clean.csv')

In [3]:
test = pd.read_csv('clean_test.csv')

In [4]:
train.drop('Unnamed: 0', axis=1, inplace=True)
test.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
train['train']=1
test['train']=0

In [6]:
combined = pd.concat([train,test])

/home/jonric/.virtualenvs/keras-tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [7]:
combined['funder'] = pd.factorize(combined['funder'])[0]
combined['scheme_management'] = pd.factorize(combined['scheme_management'])[0]
combined['extraction_type'] = pd.factorize(combined['extraction_type'])[0]
combined['management'] = pd.factorize(combined['management'])[0]
combined['payment_type'] = pd.factorize(combined['payment_type'])[0]
combined['water_quality'] = pd.factorize(combined['water_quality'])[0]
combined['quantity'] = pd.factorize(combined['quantity'])[0]
combined['source'] = pd.factorize(combined['source'])[0]
combined['waterpoint_type'] = pd.factorize(combined['waterpoint_type'])[0]
combined['basin'] = pd.factorize(combined['basin'])[0]
combined['region'] = pd.factorize(combined['region'])[0]
combined['lga'] = pd.factorize(combined['lga'])[0]
combined['district_code'] = pd.factorize(combined['district_code'])[0]
combined['operational_year'] = pd.factorize(combined['operational_year'])[0]
combined['status_group'], labels = pd.factorize(combined['status_group'])

In [8]:
train_df = combined[combined["train"] == 1]
test_df = combined[combined["train"] == 0]
train_df.drop(["train"], axis=1, inplace=True)
train_df.drop(['id'],axis=1, inplace=True)
test_df.drop(["train"], axis=1, inplace=True)

test_df_id = test_df.pop('id')
test_df_statgrp = test_df.pop('status_group')

# train = train_df
train = train_df.sample(frac=0.9)
validate = train_df.sample(frac=0.1)
#test = train_df.sample(frac=0.1)


train_target = train.pop('status_group')
validate_target = validate.pop('status_group')
#test_target = test.pop('status_group')
[xsize, ysize] =train.shape


/home/jonric/.virtualenvs/keras-tf-gpu/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from keras.optimizers import SGD, Adam, RMSprop, Nadam

num_labels = len(np.unique(train_target))
train_target_cat = to_categorical(train_target)
validate_target_cat = to_categorical(validate_target)
#test_target_cat = to_categorical(test_target)

def lr_schedule(epoch):
    lr = 1e-3
    n=60
    if epoch > 4*n:
        lr *= 0.5e-3
    elif epoch > 3*n:
        lr *= 1e-3
    elif epoch > 2*n:
        lr *= 1e-2
    elif epoch > 1*n:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

batch_size = 128
dropout = 0.1
input_size = (1 * ysize * 1)

model = Sequential()
model.add(Dense(512, input_dim=input_size))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(dropout))
# model.add(Dense(512))
# model.add(Activation('relu'))
# model.add(Dropout(dropout))
model.add(Dense(num_labels))
model.add(Activation('softmax'))


Using TensorFlow backend.


In [10]:
model.summary()

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [lr_reducer, lr_scheduler]

optim = Adam(lr=lr_schedule(0))

model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])

# model.fit(train, train_target_cat,  validation_data=(validate, validate_target_cat), epochs=300, batch_size=batch_size, callbacks = callbacks)
model.fit(train, train_target_cat, epochs=300, batch_size=batch_size, callbacks = callbacks)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               10240     
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
__________

/home/jonric/.virtualenvs/keras-tf-gpu/lib/python3.6/site-packages/keras/callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


53460/53460 [==============================] - 4s 68us/step - loss: 0.8018 - acc: 0.6270
Epoch 3/300
Learning rate:  0.001
53460/53460 [==============================] - 4s 69us/step - loss: 0.7772 - acc: 0.6444
Epoch 4/300
Learning rate:  0.001
53460/53460 [==============================] - 4s 69us/step - loss: 0.7556 - acc: 0.6582
Epoch 5/300
Learning rate:  0.001
53460/53460 [==============================] - 4s 69us/step - loss: 0.7395 - acc: 0.6687
Epoch 6/300
Learning rate:  0.001
53460/53460 [==============================] - 4s 70us/step - loss: 0.7171 - acc: 0.6813
Epoch 7/300
Learning rate:  0.001
53460/53460 [==============================] - 4s 69us/step - loss: 0.6961 - acc: 0.6930
Epoch 8/300
Learning rate:  0.001
53460/53460 [==============================] - 4s 71us/step - loss: 0.6739 - acc: 0.7064
Epoch 9/300
Learning rate:  0.001
53460/53460 [==============================] - 4s 69us/step - loss: 0.6568 - acc: 0.7150
Epoch 10/300
Learning rate:  0.001
53460/53460 [==

53460/53460 [==============================] - 4s 66us/step - loss: 0.4077 - acc: 0.8233
Epoch 69/300
Learning rate:  0.0001
53460/53460 [==============================] - 4s 66us/step - loss: 0.4055 - acc: 0.8242
Epoch 70/300
Learning rate:  0.0001
53460/53460 [==============================] - 4s 66us/step - loss: 0.4060 - acc: 0.8248
Epoch 71/300
Learning rate:  0.0001
53460/53460 [==============================] - 4s 66us/step - loss: 0.4028 - acc: 0.8254
Epoch 72/300
Learning rate:  0.0001
53460/53460 [==============================] - 4s 66us/step - loss: 0.4020 - acc: 0.8266
Epoch 73/300
Learning rate:  0.0001
53460/53460 [==============================] - 4s 66us/step - loss: 0.4012 - acc: 0.8255
Epoch 74/300
Learning rate:  0.0001
53460/53460 [==============================] - 4s 66us/step - loss: 0.4008 - acc: 0.8254
Epoch 75/300
Learning rate:  0.0001
53460/53460 [==============================] - 4s 66us/step - loss: 0.3992 - acc: 0.8266
Epoch 76/300
Learning rate:  0.0001


53460/53460 [==============================] - 4s 68us/step - loss: 0.3669 - acc: 0.8401
Epoch 134/300
Learning rate:  1e-05
53460/53460 [==============================] - 4s 69us/step - loss: 0.3655 - acc: 0.8403
Epoch 135/300
Learning rate:  1e-05
53460/53460 [==============================] - 4s 68us/step - loss: 0.3664 - acc: 0.8398
Epoch 136/300
Learning rate:  1e-05
53460/53460 [==============================] - 4s 68us/step - loss: 0.3675 - acc: 0.8401
Epoch 137/300
Learning rate:  1e-05
53460/53460 [==============================] - 4s 71us/step - loss: 0.3661 - acc: 0.8421
Epoch 138/300
Learning rate:  1e-05
53460/53460 [==============================] - 4s 69us/step - loss: 0.3661 - acc: 0.8412
Epoch 139/300
Learning rate:  1e-05
53460/53460 [==============================] - 4s 68us/step - loss: 0.3666 - acc: 0.8408
Epoch 140/300
Learning rate:  1e-05
53460/53460 [==============================] - 4s 69us/step - loss: 0.3652 - acc: 0.8410
Epoch 141/300
Learning rate:  1e-05


53460/53460 [==============================] - 4s 67us/step - loss: 0.3633 - acc: 0.8415
Epoch 199/300
Learning rate:  1e-06
53460/53460 [==============================] - 4s 67us/step - loss: 0.3616 - acc: 0.8427
Epoch 200/300
Learning rate:  1e-06
53460/53460 [==============================] - 4s 67us/step - loss: 0.3635 - acc: 0.8423
Epoch 201/300
Learning rate:  1e-06
53460/53460 [==============================] - 4s 69us/step - loss: 0.3610 - acc: 0.8420
Epoch 202/300
Learning rate:  1e-06
53460/53460 [==============================] - 4s 68us/step - loss: 0.3643 - acc: 0.8410
Epoch 203/300
Learning rate:  1e-06
53460/53460 [==============================] - 4s 68us/step - loss: 0.3654 - acc: 0.8401
Epoch 204/300
Learning rate:  1e-06
53460/53460 [==============================] - 4s 68us/step - loss: 0.3616 - acc: 0.8435
Epoch 205/300
Learning rate:  1e-06
53460/53460 [==============================] - 4s 67us/step - loss: 0.3620 - acc: 0.8429
Epoch 206/300
Learning rate:  1e-06


53460/53460 [==============================] - 4s 67us/step - loss: 0.3625 - acc: 0.8423
Epoch 264/300
Learning rate:  5e-07
53460/53460 [==============================] - 4s 68us/step - loss: 0.3625 - acc: 0.8418
Epoch 265/300
Learning rate:  5e-07
53460/53460 [==============================] - 4s 67us/step - loss: 0.3633 - acc: 0.8425
Epoch 266/300
Learning rate:  5e-07
53460/53460 [==============================] - 4s 67us/step - loss: 0.3614 - acc: 0.8418
Epoch 267/300
Learning rate:  5e-07
53460/53460 [==============================] - 4s 67us/step - loss: 0.3643 - acc: 0.8419
Epoch 268/300
Learning rate:  5e-07
53460/53460 [==============================] - 4s 68us/step - loss: 0.3626 - acc: 0.8422
Epoch 269/300
Learning rate:  5e-07
53460/53460 [==============================] - 4s 67us/step - loss: 0.3631 - acc: 0.8428
Epoch 270/300
Learning rate:  5e-07
53460/53460 [==============================] - 4s 67us/step - loss: 0.3627 - acc: 0.8419
Epoch 271/300
Learning rate:  5e-07


In [11]:
score = model.evaluate(validate, validate_target_cat, batch_size=batch_size)
print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))

5940/5940 [==============================] - 0s 18us/step

Test accuracy: 85.5%


In [12]:
test_df_result = model.predict_classes(test_df)
# for i in range(test_df_result.shape[-1]):
#     print(test_df_id[i],labels[test_df_result[i]])

In [13]:
test_df_pred = pd.DataFrame(labels[test_df_result])
test_df_pred.columns = ['status_group']
test_df_id = pd.DataFrame(test_df_id)
test_df_pred = test_df_id.join(test_df_pred)

In [14]:
test_df_pred.to_csv("submission.csv", index=False)